<a href="https://colab.research.google.com/github/Ravikumarchavva/Ravikumar/blob/main/ipl/ipl_full_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [129]:
import polars as pl
import numpy as np
import matplotlib.pyplot as mp
from pycaret.regression import *

In [130]:
matches=pl.read_csv("https://raw.githubusercontent.com/Ravikumarchavva/Ravikumar/main/ipl/matches.csv")
matches

match_id,season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
i64,i64,str,str,str,str,str,str,str,i64,str,i64,i64,str,str,str,str,str
1,2017,"""Hyderabad""","""2017-04-05""","""Sunrisers Hyde…","""Royal Challeng…","""Royal Challeng…","""field""","""normal""",0,"""Sunrisers Hyde…",35,0,"""Yuvraj Singh""","""Rajiv Gandhi I…","""AY Dandekar""","""NJ Llong""",null
2,2017,"""Pune""","""2017-04-06""","""Mumbai Indians…","""Rising Pune Su…","""Rising Pune Su…","""field""","""normal""",0,"""Rising Pune Su…",0,7,"""SPD Smith""","""Maharashtra Cr…","""A Nand Kishore…","""S Ravi""",null
3,2017,"""Rajkot""","""2017-04-07""","""Gujarat Lions""","""Kolkata Knight…","""Kolkata Knight…","""field""","""normal""",0,"""Kolkata Knight…",0,10,"""CA Lynn""","""Saurashtra Cri…","""Nitin Menon""","""CK Nandan""",null
4,2017,"""Indore""","""2017-04-08""","""Rising Pune Su…","""Kings XI Punja…","""Kings XI Punja…","""field""","""normal""",0,"""Kings XI Punja…",0,6,"""GJ Maxwell""","""Holkar Cricket…","""AK Chaudhary""","""C Shamshuddin""",null
5,2017,"""Bangalore""","""2017-04-08""","""Royal Challeng…","""Delhi Daredevi…","""Royal Challeng…","""bat""","""normal""",0,"""Royal Challeng…",15,0,"""KM Jadhav""","""M Chinnaswamy …",null,null,null
6,2017,"""Hyderabad""","""2017-04-09""","""Gujarat Lions""","""Sunrisers Hyde…","""Sunrisers Hyde…","""field""","""normal""",0,"""Sunrisers Hyde…",0,9,"""Rashid Khan""","""Rajiv Gandhi I…","""A Deshmukh""","""NJ Llong""",null
7,2017,"""Mumbai""","""2017-04-09""","""Kolkata Knight…","""Mumbai Indians…","""Mumbai Indians…","""field""","""normal""",0,"""Mumbai Indians…",0,4,"""N Rana""","""Wankhede Stadi…","""Nitin Menon""","""CK Nandan""",null
8,2017,"""Indore""","""2017-04-10""","""Royal Challeng…","""Kings XI Punja…","""Royal Challeng…","""bat""","""normal""",0,"""Kings XI Punja…",0,8,"""AR Patel""","""Holkar Cricket…","""AK Chaudhary""","""C Shamshuddin""",null
9,2017,"""Pune""","""2017-04-11""","""Delhi Daredevi…","""Rising Pune Su…","""Rising Pune Su…","""field""","""normal""",0,"""Delhi Daredevi…",97,0,"""SV Samson""","""Maharashtra Cr…","""AY Dandekar""","""S Ravi""",null


In [131]:
matches=matches.drop("season","player_of_match","umpire1","umpire2",'umpire3',"win_by_runs","win_by_wickets").filter(pl.col("dl_applied")==0)
matches

match_id,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,venue
i64,str,str,str,str,str,str,str,i64,str,str
1,"""Hyderabad""","""2017-04-05""","""Sunrisers Hyde…","""Royal Challeng…","""Royal Challeng…","""field""","""normal""",0,"""Sunrisers Hyde…","""Rajiv Gandhi I…"
2,"""Pune""","""2017-04-06""","""Mumbai Indians…","""Rising Pune Su…","""Rising Pune Su…","""field""","""normal""",0,"""Rising Pune Su…","""Maharashtra Cr…"
3,"""Rajkot""","""2017-04-07""","""Gujarat Lions""","""Kolkata Knight…","""Kolkata Knight…","""field""","""normal""",0,"""Kolkata Knight…","""Saurashtra Cri…"
4,"""Indore""","""2017-04-08""","""Rising Pune Su…","""Kings XI Punja…","""Kings XI Punja…","""field""","""normal""",0,"""Kings XI Punja…","""Holkar Cricket…"
5,"""Bangalore""","""2017-04-08""","""Royal Challeng…","""Delhi Daredevi…","""Royal Challeng…","""bat""","""normal""",0,"""Royal Challeng…","""M Chinnaswamy …"
6,"""Hyderabad""","""2017-04-09""","""Gujarat Lions""","""Sunrisers Hyde…","""Sunrisers Hyde…","""field""","""normal""",0,"""Sunrisers Hyde…","""Rajiv Gandhi I…"
7,"""Mumbai""","""2017-04-09""","""Kolkata Knight…","""Mumbai Indians…","""Mumbai Indians…","""field""","""normal""",0,"""Mumbai Indians…","""Wankhede Stadi…"
8,"""Indore""","""2017-04-10""","""Royal Challeng…","""Kings XI Punja…","""Royal Challeng…","""bat""","""normal""",0,"""Kings XI Punja…","""Holkar Cricket…"
9,"""Pune""","""2017-04-11""","""Delhi Daredevi…","""Rising Pune Su…","""Rising Pune Su…","""field""","""normal""",0,"""Delhi Daredevi…","""Maharashtra Cr…"


In [132]:
matches=matches.filter(pl.col("result")=="normal").drop("result","dl_applied","date")
matches

match_id,city,team1,team2,toss_winner,toss_decision,winner,venue
i64,str,str,str,str,str,str,str
1,"""Hyderabad""","""Sunrisers Hyde…","""Royal Challeng…","""Royal Challeng…","""field""","""Sunrisers Hyde…","""Rajiv Gandhi I…"
2,"""Pune""","""Mumbai Indians…","""Rising Pune Su…","""Rising Pune Su…","""field""","""Rising Pune Su…","""Maharashtra Cr…"
3,"""Rajkot""","""Gujarat Lions""","""Kolkata Knight…","""Kolkata Knight…","""field""","""Kolkata Knight…","""Saurashtra Cri…"
4,"""Indore""","""Rising Pune Su…","""Kings XI Punja…","""Kings XI Punja…","""field""","""Kings XI Punja…","""Holkar Cricket…"
5,"""Bangalore""","""Royal Challeng…","""Delhi Daredevi…","""Royal Challeng…","""bat""","""Royal Challeng…","""M Chinnaswamy …"
6,"""Hyderabad""","""Gujarat Lions""","""Sunrisers Hyde…","""Sunrisers Hyde…","""field""","""Sunrisers Hyde…","""Rajiv Gandhi I…"
7,"""Mumbai""","""Kolkata Knight…","""Mumbai Indians…","""Mumbai Indians…","""field""","""Mumbai Indians…","""Wankhede Stadi…"
8,"""Indore""","""Royal Challeng…","""Kings XI Punja…","""Royal Challeng…","""bat""","""Kings XI Punja…","""Holkar Cricket…"
9,"""Pune""","""Delhi Daredevi…","""Rising Pune Su…","""Rising Pune Su…","""field""","""Delhi Daredevi…","""Maharashtra Cr…"


In [133]:
deliveries=pl.read_csv("https://raw.githubusercontent.com/Ravikumarchavva/Ravikumar/main/ipl/deliveries.csv")
deliveries=deliveries.filter(pl.col("match_id").is_in(matches["match_id"]))
deliveries

match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,wide_runs,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder
i64,i64,str,str,i64,i64,str,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,str,str
1,1,"""Sunrisers Hyde…","""Royal Challeng…",1,1,"""DA Warner""","""S Dhawan""","""TS Mills""",0,0,0,0,0,0,0,0,0,null,null,null
1,1,"""Sunrisers Hyde…","""Royal Challeng…",1,2,"""DA Warner""","""S Dhawan""","""TS Mills""",0,0,0,0,0,0,0,0,0,null,null,null
1,1,"""Sunrisers Hyde…","""Royal Challeng…",1,3,"""DA Warner""","""S Dhawan""","""TS Mills""",0,0,0,0,0,0,4,0,4,null,null,null
1,1,"""Sunrisers Hyde…","""Royal Challeng…",1,4,"""DA Warner""","""S Dhawan""","""TS Mills""",0,0,0,0,0,0,0,0,0,null,null,null
1,1,"""Sunrisers Hyde…","""Royal Challeng…",1,5,"""DA Warner""","""S Dhawan""","""TS Mills""",0,2,0,0,0,0,0,2,2,null,null,null
1,1,"""Sunrisers Hyde…","""Royal Challeng…",1,6,"""S Dhawan""","""DA Warner""","""TS Mills""",0,0,0,0,0,0,0,0,0,null,null,null
1,1,"""Sunrisers Hyde…","""Royal Challeng…",1,7,"""S Dhawan""","""DA Warner""","""TS Mills""",0,0,0,1,0,0,0,1,1,null,null,null
1,1,"""Sunrisers Hyde…","""Royal Challeng…",2,1,"""S Dhawan""","""DA Warner""","""A Choudhary""",0,0,0,0,0,0,1,0,1,null,null,null
1,1,"""Sunrisers Hyde…","""Royal Challeng…",2,2,"""DA Warner""","""S Dhawan""","""A Choudhary""",0,0,0,0,0,0,4,0,4,null,null,null


In [134]:
deliveries=deliveries.drop("batsman","non_striker",	"bowler","is_super_over","dismissal_kind","fielder","wide_runs"	,"bye_runs","legbye_runs","noball_runs","penalty_runs","batsman_runs","extra_runs")
deliveries

match_id,inning,batting_team,bowling_team,over,ball,total_runs,player_dismissed
i64,i64,str,str,i64,i64,i64,str
1,1,"""Sunrisers Hyde…","""Royal Challeng…",1,1,0,null
1,1,"""Sunrisers Hyde…","""Royal Challeng…",1,2,0,null
1,1,"""Sunrisers Hyde…","""Royal Challeng…",1,3,4,null
1,1,"""Sunrisers Hyde…","""Royal Challeng…",1,4,0,null
1,1,"""Sunrisers Hyde…","""Royal Challeng…",1,5,2,null
1,1,"""Sunrisers Hyde…","""Royal Challeng…",1,6,0,null
1,1,"""Sunrisers Hyde…","""Royal Challeng…",1,7,1,null
1,1,"""Sunrisers Hyde…","""Royal Challeng…",2,1,1,null
1,1,"""Sunrisers Hyde…","""Royal Challeng…",2,2,4,null


In [135]:
deliveries=deliveries.with_columns(pl.col("player_dismissed").apply(lambda value : 0 if value is None else 1)).fill_null(0)
deliveries

match_id,inning,batting_team,bowling_team,over,ball,total_runs,player_dismissed
i64,i64,str,str,i64,i64,i64,i64
1,1,"""Sunrisers Hyde…","""Royal Challeng…",1,1,0,0
1,1,"""Sunrisers Hyde…","""Royal Challeng…",1,2,0,0
1,1,"""Sunrisers Hyde…","""Royal Challeng…",1,3,4,0
1,1,"""Sunrisers Hyde…","""Royal Challeng…",1,4,0,0
1,1,"""Sunrisers Hyde…","""Royal Challeng…",1,5,2,0
1,1,"""Sunrisers Hyde…","""Royal Challeng…",1,6,0,0
1,1,"""Sunrisers Hyde…","""Royal Challeng…",1,7,1,0
1,1,"""Sunrisers Hyde…","""Royal Challeng…",2,1,1,0
1,1,"""Sunrisers Hyde…","""Royal Challeng…",2,2,4,0


In [136]:
group_data=deliveries.groupby(pl.col("match_id"),pl.col("inning"),pl.col("over")).agg((pl.col("total_runs").sum()),(pl.col("player_dismissed").sum())).fill_null(0)
group_data

match_id,inning,over,total_runs,player_dismissed
i64,i64,i64,i64,i64
1,1,4,4,0
1,2,1,11,0
1,2,20,7,1
2,1,12,3,1
2,1,19,9,1
2,1,20,30,1
3,1,14,7,0
3,1,19,16,0
3,2,5,7,0


In [137]:
final_score_data=group_data.groupby(["match_id", "inning"]).agg(pl.col("total_runs").sum().alias("final_runs")).sort("match_id")
final_score_data

match_id,inning,final_runs
i64,i64,i64
1,1,207
1,2,172
2,1,184
2,2,187
3,2,184
3,1,183
4,2,164
4,1,163
5,1,157


In [138]:
deliveries

match_id,inning,batting_team,bowling_team,over,ball,total_runs,player_dismissed
i64,i64,str,str,i64,i64,i64,i64
1,1,"""Sunrisers Hyde…","""Royal Challeng…",1,1,0,0
1,1,"""Sunrisers Hyde…","""Royal Challeng…",1,2,0,0
1,1,"""Sunrisers Hyde…","""Royal Challeng…",1,3,4,0
1,1,"""Sunrisers Hyde…","""Royal Challeng…",1,4,0,0
1,1,"""Sunrisers Hyde…","""Royal Challeng…",1,5,2,0
1,1,"""Sunrisers Hyde…","""Royal Challeng…",1,6,0,0
1,1,"""Sunrisers Hyde…","""Royal Challeng…",1,7,1,0
1,1,"""Sunrisers Hyde…","""Royal Challeng…",2,1,1,0
1,1,"""Sunrisers Hyde…","""Royal Challeng…",2,2,4,0


In [139]:
deliveries=deliveries.join(final_score_data,on=['match_id',"inning"]).join(matches,on=['match_id'])
deliveries

match_id,inning,batting_team,bowling_team,over,ball,total_runs,player_dismissed,final_runs,city,team1,team2,toss_winner,toss_decision,winner,venue
i64,i64,str,str,i64,i64,i64,i64,i64,str,str,str,str,str,str,str
1,1,"""Sunrisers Hyde…","""Royal Challeng…",1,1,0,0,207,"""Hyderabad""","""Sunrisers Hyde…","""Royal Challeng…","""Royal Challeng…","""field""","""Sunrisers Hyde…","""Rajiv Gandhi I…"
1,1,"""Sunrisers Hyde…","""Royal Challeng…",1,2,0,0,207,"""Hyderabad""","""Sunrisers Hyde…","""Royal Challeng…","""Royal Challeng…","""field""","""Sunrisers Hyde…","""Rajiv Gandhi I…"
1,1,"""Sunrisers Hyde…","""Royal Challeng…",1,3,4,0,207,"""Hyderabad""","""Sunrisers Hyde…","""Royal Challeng…","""Royal Challeng…","""field""","""Sunrisers Hyde…","""Rajiv Gandhi I…"
1,1,"""Sunrisers Hyde…","""Royal Challeng…",1,4,0,0,207,"""Hyderabad""","""Sunrisers Hyde…","""Royal Challeng…","""Royal Challeng…","""field""","""Sunrisers Hyde…","""Rajiv Gandhi I…"
1,1,"""Sunrisers Hyde…","""Royal Challeng…",1,5,2,0,207,"""Hyderabad""","""Sunrisers Hyde…","""Royal Challeng…","""Royal Challeng…","""field""","""Sunrisers Hyde…","""Rajiv Gandhi I…"
1,1,"""Sunrisers Hyde…","""Royal Challeng…",1,6,0,0,207,"""Hyderabad""","""Sunrisers Hyde…","""Royal Challeng…","""Royal Challeng…","""field""","""Sunrisers Hyde…","""Rajiv Gandhi I…"
1,1,"""Sunrisers Hyde…","""Royal Challeng…",1,7,1,0,207,"""Hyderabad""","""Sunrisers Hyde…","""Royal Challeng…","""Royal Challeng…","""field""","""Sunrisers Hyde…","""Rajiv Gandhi I…"
1,1,"""Sunrisers Hyde…","""Royal Challeng…",2,1,1,0,207,"""Hyderabad""","""Sunrisers Hyde…","""Royal Challeng…","""Royal Challeng…","""field""","""Sunrisers Hyde…","""Rajiv Gandhi I…"
1,1,"""Sunrisers Hyde…","""Royal Challeng…",2,2,4,0,207,"""Hyderabad""","""Sunrisers Hyde…","""Royal Challeng…","""Royal Challeng…","""field""","""Sunrisers Hyde…","""Rajiv Gandhi I…"


In [140]:
deliveries=deliveries.with_columns(((pl.col("over")+pl.col("ball")/10)-1).alias("balls")).drop("ball","over")
deliveries

match_id,inning,batting_team,bowling_team,total_runs,player_dismissed,final_runs,city,team1,team2,toss_winner,toss_decision,winner,venue,balls
i64,i64,str,str,i64,i64,i64,str,str,str,str,str,str,str,f64
1,1,"""Sunrisers Hyde…","""Royal Challeng…",0,0,207,"""Hyderabad""","""Sunrisers Hyde…","""Royal Challeng…","""Royal Challeng…","""field""","""Sunrisers Hyde…","""Rajiv Gandhi I…",0.1
1,1,"""Sunrisers Hyde…","""Royal Challeng…",0,0,207,"""Hyderabad""","""Sunrisers Hyde…","""Royal Challeng…","""Royal Challeng…","""field""","""Sunrisers Hyde…","""Rajiv Gandhi I…",0.2
1,1,"""Sunrisers Hyde…","""Royal Challeng…",4,0,207,"""Hyderabad""","""Sunrisers Hyde…","""Royal Challeng…","""Royal Challeng…","""field""","""Sunrisers Hyde…","""Rajiv Gandhi I…",0.3
1,1,"""Sunrisers Hyde…","""Royal Challeng…",0,0,207,"""Hyderabad""","""Sunrisers Hyde…","""Royal Challeng…","""Royal Challeng…","""field""","""Sunrisers Hyde…","""Rajiv Gandhi I…",0.4
1,1,"""Sunrisers Hyde…","""Royal Challeng…",2,0,207,"""Hyderabad""","""Sunrisers Hyde…","""Royal Challeng…","""Royal Challeng…","""field""","""Sunrisers Hyde…","""Rajiv Gandhi I…",0.5
1,1,"""Sunrisers Hyde…","""Royal Challeng…",0,0,207,"""Hyderabad""","""Sunrisers Hyde…","""Royal Challeng…","""Royal Challeng…","""field""","""Sunrisers Hyde…","""Rajiv Gandhi I…",0.6
1,1,"""Sunrisers Hyde…","""Royal Challeng…",1,0,207,"""Hyderabad""","""Sunrisers Hyde…","""Royal Challeng…","""Royal Challeng…","""field""","""Sunrisers Hyde…","""Rajiv Gandhi I…",0.7
1,1,"""Sunrisers Hyde…","""Royal Challeng…",1,0,207,"""Hyderabad""","""Sunrisers Hyde…","""Royal Challeng…","""Royal Challeng…","""field""","""Sunrisers Hyde…","""Rajiv Gandhi I…",1.1
1,1,"""Sunrisers Hyde…","""Royal Challeng…",4,0,207,"""Hyderabad""","""Sunrisers Hyde…","""Royal Challeng…","""Royal Challeng…","""field""","""Sunrisers Hyde…","""Rajiv Gandhi I…",1.2


In [141]:
deliveries=deliveries.filter(pl.col("team1")!="Kochi Tuskers Kerala").filter(pl.col("team1")!="Deccan Chargers").filter(pl.col("team2")!="Kochi Tuskers Kerala").filter(pl.col("team2")!="Deccan Chargers")
deliveries

match_id,inning,batting_team,bowling_team,total_runs,player_dismissed,final_runs,city,team1,team2,toss_winner,toss_decision,winner,venue,balls
i64,i64,str,str,i64,i64,i64,str,str,str,str,str,str,str,f64
1,1,"""Sunrisers Hyde…","""Royal Challeng…",0,0,207,"""Hyderabad""","""Sunrisers Hyde…","""Royal Challeng…","""Royal Challeng…","""field""","""Sunrisers Hyde…","""Rajiv Gandhi I…",0.1
1,1,"""Sunrisers Hyde…","""Royal Challeng…",0,0,207,"""Hyderabad""","""Sunrisers Hyde…","""Royal Challeng…","""Royal Challeng…","""field""","""Sunrisers Hyde…","""Rajiv Gandhi I…",0.2
1,1,"""Sunrisers Hyde…","""Royal Challeng…",4,0,207,"""Hyderabad""","""Sunrisers Hyde…","""Royal Challeng…","""Royal Challeng…","""field""","""Sunrisers Hyde…","""Rajiv Gandhi I…",0.3
1,1,"""Sunrisers Hyde…","""Royal Challeng…",0,0,207,"""Hyderabad""","""Sunrisers Hyde…","""Royal Challeng…","""Royal Challeng…","""field""","""Sunrisers Hyde…","""Rajiv Gandhi I…",0.4
1,1,"""Sunrisers Hyde…","""Royal Challeng…",2,0,207,"""Hyderabad""","""Sunrisers Hyde…","""Royal Challeng…","""Royal Challeng…","""field""","""Sunrisers Hyde…","""Rajiv Gandhi I…",0.5
1,1,"""Sunrisers Hyde…","""Royal Challeng…",0,0,207,"""Hyderabad""","""Sunrisers Hyde…","""Royal Challeng…","""Royal Challeng…","""field""","""Sunrisers Hyde…","""Rajiv Gandhi I…",0.6
1,1,"""Sunrisers Hyde…","""Royal Challeng…",1,0,207,"""Hyderabad""","""Sunrisers Hyde…","""Royal Challeng…","""Royal Challeng…","""field""","""Sunrisers Hyde…","""Rajiv Gandhi I…",0.7
1,1,"""Sunrisers Hyde…","""Royal Challeng…",1,0,207,"""Hyderabad""","""Sunrisers Hyde…","""Royal Challeng…","""Royal Challeng…","""field""","""Sunrisers Hyde…","""Rajiv Gandhi I…",1.1
1,1,"""Sunrisers Hyde…","""Royal Challeng…",4,0,207,"""Hyderabad""","""Sunrisers Hyde…","""Royal Challeng…","""Royal Challeng…","""field""","""Sunrisers Hyde…","""Rajiv Gandhi I…",1.2


In [142]:
deliveries=deliveries.with_columns(pl.col("batting_team").str.replace("Rising Pune Supergiant","Pune Warriors"))
deliveries=deliveries.with_columns(pl.col("bowling_team").str.replace("Rising Pune Supergiant","Pune Warriors"))
deliveries=deliveries.with_columns(pl.col("team1").str.replace("Rising Pune Supergiant","Pune Warriors"))
deliveries=deliveries.with_columns(pl.col("team2").str.replace("Rising Pune Supergiant","Pune Warriors"))
deliveries=deliveries.with_columns(pl.col("toss_winner").str.replace("Rising Pune Supergiant","Pune Warriors"))

deliveries=deliveries.with_columns(pl.col("batting_team").str.replace("Rising Pune Supergiants","Pune Warriors"))
deliveries=deliveries.with_columns(pl.col("bowling_team").str.replace("Rising Pune Supergiants","Pune Warriors"))
deliveries=deliveries.with_columns(pl.col("team1").str.replace("Rising Pune Supergiants","Pune Warriors"))
deliveries=deliveries.with_columns(pl.col("team2").str.replace("Rising Pune Supergiants","Pune Warriors"))
deliveries=deliveries.with_columns(pl.col("toss_winner").str.replace("Rising Pune Supergiants","Pune Warriors"))

deliveries=deliveries.with_columns(pl.col("batting_team").str.replace("Pune Warriorss","Pune Warriors"))
deliveries=deliveries.with_columns(pl.col("bowling_team").str.replace("Pune Warriorss","Pune Warriors"))
deliveries=deliveries.with_columns(pl.col("team1").str.replace("Pune Warriorss","Pune Warriors"))
deliveries=deliveries.with_columns(pl.col("team2").str.replace("Pune Warriorss","Pune Warriors"))
deliveries=deliveries.with_columns(pl.col("toss_winner").str.replace("Pune Warriorss","Pune Warriors"))

deliveries=deliveries.with_columns(pl.col("batting_team").str.replace("Delhi Daredevils","Delhi Capitals"))
deliveries=deliveries.with_columns(pl.col("bowling_team").str.replace("Delhi Daredevils","Delhi Capitals"))
deliveries=deliveries.with_columns(pl.col("team1").str.replace("Delhi Daredevils","Delhi Capitals"))
deliveries=deliveries.with_columns(pl.col("team2").str.replace("Delhi Daredevils","Delhi Capitals"))
deliveries=deliveries.with_columns(pl.col("toss_winner").str.replace("Delhi Daredevils","Delhi Capitals"))


deliveries=deliveries.with_columns(pl.col("toss_decision").str.replace("field",1)).with_columns(pl.col("toss_decision").str.replace("bat",0))

In [143]:
deliveries=deliveries.with_columns(deliveries["toss_decision"].cast(pl.Int64()))
deliveries

match_id,inning,batting_team,bowling_team,total_runs,player_dismissed,final_runs,city,team1,team2,toss_winner,toss_decision,winner,venue,balls
i64,i64,str,str,i64,i64,i64,str,str,str,str,i64,str,str,f64
1,1,"""Sunrisers Hyde…","""Royal Challeng…",0,0,207,"""Hyderabad""","""Sunrisers Hyde…","""Royal Challeng…","""Royal Challeng…",1,"""Sunrisers Hyde…","""Rajiv Gandhi I…",0.1
1,1,"""Sunrisers Hyde…","""Royal Challeng…",0,0,207,"""Hyderabad""","""Sunrisers Hyde…","""Royal Challeng…","""Royal Challeng…",1,"""Sunrisers Hyde…","""Rajiv Gandhi I…",0.2
1,1,"""Sunrisers Hyde…","""Royal Challeng…",4,0,207,"""Hyderabad""","""Sunrisers Hyde…","""Royal Challeng…","""Royal Challeng…",1,"""Sunrisers Hyde…","""Rajiv Gandhi I…",0.3
1,1,"""Sunrisers Hyde…","""Royal Challeng…",0,0,207,"""Hyderabad""","""Sunrisers Hyde…","""Royal Challeng…","""Royal Challeng…",1,"""Sunrisers Hyde…","""Rajiv Gandhi I…",0.4
1,1,"""Sunrisers Hyde…","""Royal Challeng…",2,0,207,"""Hyderabad""","""Sunrisers Hyde…","""Royal Challeng…","""Royal Challeng…",1,"""Sunrisers Hyde…","""Rajiv Gandhi I…",0.5
1,1,"""Sunrisers Hyde…","""Royal Challeng…",0,0,207,"""Hyderabad""","""Sunrisers Hyde…","""Royal Challeng…","""Royal Challeng…",1,"""Sunrisers Hyde…","""Rajiv Gandhi I…",0.6
1,1,"""Sunrisers Hyde…","""Royal Challeng…",1,0,207,"""Hyderabad""","""Sunrisers Hyde…","""Royal Challeng…","""Royal Challeng…",1,"""Sunrisers Hyde…","""Rajiv Gandhi I…",0.7
1,1,"""Sunrisers Hyde…","""Royal Challeng…",1,0,207,"""Hyderabad""","""Sunrisers Hyde…","""Royal Challeng…","""Royal Challeng…",1,"""Sunrisers Hyde…","""Rajiv Gandhi I…",1.1
1,1,"""Sunrisers Hyde…","""Royal Challeng…",4,0,207,"""Hyderabad""","""Sunrisers Hyde…","""Royal Challeng…","""Royal Challeng…",1,"""Sunrisers Hyde…","""Rajiv Gandhi I…",1.2


In [144]:
# window function and cumulative sum
deliveries=deliveries.with_columns(pl.col("total_runs").cumsum().over(deliveries["match_id"],deliveries["inning"]).alias("current_runs")).drop("total_runs").fill_null(0)
deliveries=deliveries.with_columns(pl.col("player_dismissed").cumsum().over(deliveries["match_id"],deliveries["inning"]).alias("wickets")).drop("player_dismissed")
deliveries

match_id,inning,batting_team,bowling_team,final_runs,city,team1,team2,toss_winner,toss_decision,winner,venue,balls,current_runs,wickets
i64,i64,str,str,i64,str,str,str,str,i64,str,str,f64,i64,i64
1,1,"""Sunrisers Hyde…","""Royal Challeng…",207,"""Hyderabad""","""Sunrisers Hyde…","""Royal Challeng…","""Royal Challeng…",1,"""Sunrisers Hyde…","""Rajiv Gandhi I…",0.1,0,0
1,1,"""Sunrisers Hyde…","""Royal Challeng…",207,"""Hyderabad""","""Sunrisers Hyde…","""Royal Challeng…","""Royal Challeng…",1,"""Sunrisers Hyde…","""Rajiv Gandhi I…",0.2,0,0
1,1,"""Sunrisers Hyde…","""Royal Challeng…",207,"""Hyderabad""","""Sunrisers Hyde…","""Royal Challeng…","""Royal Challeng…",1,"""Sunrisers Hyde…","""Rajiv Gandhi I…",0.3,4,0
1,1,"""Sunrisers Hyde…","""Royal Challeng…",207,"""Hyderabad""","""Sunrisers Hyde…","""Royal Challeng…","""Royal Challeng…",1,"""Sunrisers Hyde…","""Rajiv Gandhi I…",0.4,4,0
1,1,"""Sunrisers Hyde…","""Royal Challeng…",207,"""Hyderabad""","""Sunrisers Hyde…","""Royal Challeng…","""Royal Challeng…",1,"""Sunrisers Hyde…","""Rajiv Gandhi I…",0.5,6,0
1,1,"""Sunrisers Hyde…","""Royal Challeng…",207,"""Hyderabad""","""Sunrisers Hyde…","""Royal Challeng…","""Royal Challeng…",1,"""Sunrisers Hyde…","""Rajiv Gandhi I…",0.6,6,0
1,1,"""Sunrisers Hyde…","""Royal Challeng…",207,"""Hyderabad""","""Sunrisers Hyde…","""Royal Challeng…","""Royal Challeng…",1,"""Sunrisers Hyde…","""Rajiv Gandhi I…",0.7,7,0
1,1,"""Sunrisers Hyde…","""Royal Challeng…",207,"""Hyderabad""","""Sunrisers Hyde…","""Royal Challeng…","""Royal Challeng…",1,"""Sunrisers Hyde…","""Rajiv Gandhi I…",1.1,8,0
1,1,"""Sunrisers Hyde…","""Royal Challeng…",207,"""Hyderabad""","""Sunrisers Hyde…","""Royal Challeng…","""Royal Challeng…",1,"""Sunrisers Hyde…","""Rajiv Gandhi I…",1.2,12,0


In [145]:
deliveries.filter((pl.col("match_id")==11148 ) & (pl.col("inning")==1) & (pl.col("balls")<=19))

match_id,inning,batting_team,bowling_team,final_runs,city,team1,team2,toss_winner,toss_decision,winner,venue,balls,current_runs,wickets
i64,i64,str,str,i64,str,str,str,str,i64,str,str,f64,i64,i64
11148,1,"""Chennai Super …","""Rajasthan Roya…",182,"""Chennai""","""Chennai Super …","""Rajasthan Roya…","""Rajasthan Roya…",1,"""Chennai Super …","""M. A. Chidamba…",0.1,0,0
11148,1,"""Chennai Super …","""Rajasthan Roya…",182,"""Chennai""","""Chennai Super …","""Rajasthan Roya…","""Rajasthan Roya…",1,"""Chennai Super …","""M. A. Chidamba…",0.2,1,0
11148,1,"""Chennai Super …","""Rajasthan Roya…",182,"""Chennai""","""Chennai Super …","""Rajasthan Roya…","""Rajasthan Roya…",1,"""Chennai Super …","""M. A. Chidamba…",0.3,1,0
11148,1,"""Chennai Super …","""Rajasthan Roya…",182,"""Chennai""","""Chennai Super …","""Rajasthan Roya…","""Rajasthan Roya…",1,"""Chennai Super …","""M. A. Chidamba…",0.4,1,0
11148,1,"""Chennai Super …","""Rajasthan Roya…",182,"""Chennai""","""Chennai Super …","""Rajasthan Roya…","""Rajasthan Roya…",1,"""Chennai Super …","""M. A. Chidamba…",0.5,1,0
11148,1,"""Chennai Super …","""Rajasthan Roya…",182,"""Chennai""","""Chennai Super …","""Rajasthan Roya…","""Rajasthan Roya…",1,"""Chennai Super …","""M. A. Chidamba…",0.6,1,0
11148,1,"""Chennai Super …","""Rajasthan Roya…",182,"""Chennai""","""Chennai Super …","""Rajasthan Roya…","""Rajasthan Roya…",1,"""Chennai Super …","""M. A. Chidamba…",1.1,1,0
11148,1,"""Chennai Super …","""Rajasthan Roya…",182,"""Chennai""","""Chennai Super …","""Rajasthan Roya…","""Rajasthan Roya…",1,"""Chennai Super …","""M. A. Chidamba…",1.2,1,0
11148,1,"""Chennai Super …","""Rajasthan Roya…",182,"""Chennai""","""Chennai Super …","""Rajasthan Roya…","""Rajasthan Roya…",1,"""Chennai Super …","""M. A. Chidamba…",1.3,1,0


In [146]:
deliveries=deliveries.drop("match_id","team1","team2","winner","venue").fill_null(0)
deliveries

inning,batting_team,bowling_team,final_runs,city,toss_winner,toss_decision,balls,current_runs,wickets
i64,str,str,i64,str,str,i64,f64,i64,i64
1,"""Sunrisers Hyde…","""Royal Challeng…",207,"""Hyderabad""","""Royal Challeng…",1,0.1,0,0
1,"""Sunrisers Hyde…","""Royal Challeng…",207,"""Hyderabad""","""Royal Challeng…",1,0.2,0,0
1,"""Sunrisers Hyde…","""Royal Challeng…",207,"""Hyderabad""","""Royal Challeng…",1,0.3,4,0
1,"""Sunrisers Hyde…","""Royal Challeng…",207,"""Hyderabad""","""Royal Challeng…",1,0.4,4,0
1,"""Sunrisers Hyde…","""Royal Challeng…",207,"""Hyderabad""","""Royal Challeng…",1,0.5,6,0
1,"""Sunrisers Hyde…","""Royal Challeng…",207,"""Hyderabad""","""Royal Challeng…",1,0.6,6,0
1,"""Sunrisers Hyde…","""Royal Challeng…",207,"""Hyderabad""","""Royal Challeng…",1,0.7,7,0
1,"""Sunrisers Hyde…","""Royal Challeng…",207,"""Hyderabad""","""Royal Challeng…",1,1.1,8,0
1,"""Sunrisers Hyde…","""Royal Challeng…",207,"""Hyderabad""","""Royal Challeng…",1,1.2,12,0


In [147]:
deliveries=deliveries.with_columns((pl.col("current_runs")/(pl.col("balls").ceil())).alias("current_run_rate"))
deliveries=deliveries.filter(pl.col("city")!=None)
deliveries[0:15]

inning,batting_team,bowling_team,final_runs,city,toss_winner,toss_decision,balls,current_runs,wickets,current_run_rate
i64,str,str,i64,str,str,i64,f64,i64,i64,f64
1,"""Sunrisers Hyde…","""Royal Challeng…",207,"""Hyderabad""","""Royal Challeng…",1,0.1,0,0,0.0
1,"""Sunrisers Hyde…","""Royal Challeng…",207,"""Hyderabad""","""Royal Challeng…",1,0.2,0,0,0.0
1,"""Sunrisers Hyde…","""Royal Challeng…",207,"""Hyderabad""","""Royal Challeng…",1,0.3,4,0,4.0
1,"""Sunrisers Hyde…","""Royal Challeng…",207,"""Hyderabad""","""Royal Challeng…",1,0.4,4,0,4.0
1,"""Sunrisers Hyde…","""Royal Challeng…",207,"""Hyderabad""","""Royal Challeng…",1,0.5,6,0,6.0
1,"""Sunrisers Hyde…","""Royal Challeng…",207,"""Hyderabad""","""Royal Challeng…",1,0.6,6,0,6.0
1,"""Sunrisers Hyde…","""Royal Challeng…",207,"""Hyderabad""","""Royal Challeng…",1,0.7,7,0,7.0
1,"""Sunrisers Hyde…","""Royal Challeng…",207,"""Hyderabad""","""Royal Challeng…",1,1.1,8,0,4.0
1,"""Sunrisers Hyde…","""Royal Challeng…",207,"""Hyderabad""","""Royal Challeng…",1,1.2,12,0,6.0


In [148]:
deliveries["city"].unique().to_list()

['Dharamsala',
 'Kolkata',
 'Ranchi',
 'Raipur',
 'Mumbai',
 'Ahmedabad',
 'Chennai',
 'Kimberley',
 'Indore',
 'Durban',
 'Abu Dhabi',
 'Cape Town',
 'Visakhapatnam',
 'Hyderabad',
 'Bloemfontein',
 'Centurion',
 'Johannesburg',
 'Rajkot',
 'Kanpur',
 'Delhi',
 'East London',
 'Cuttack',
 'Port Elizabeth',
 'Sharjah',
 'Mohali',
 'Bengaluru',
 'Bangalore',
 'Pune',
 'Chandigarh',
 'Jaipur']

In [149]:
city_list=['Ahmedabad','Mumbai','Jaipur','Hyderabad','Dharamsala','Bengaluru','Kolkata','Mohali','Delhi','Chennai','Chandigarh','Pune','Bangalore']

In [150]:
deliveries=deliveries.filter((deliveries["city"].is_in(city_list)))
deliveries=deliveries.with_columns(pl.col("city").str.replace('Bengaluru','Bangalore'))
deliveries

inning,batting_team,bowling_team,final_runs,city,toss_winner,toss_decision,balls,current_runs,wickets,current_run_rate
i64,str,str,i64,str,str,i64,f64,i64,i64,f64
1,"""Sunrisers Hyde…","""Royal Challeng…",207,"""Hyderabad""","""Royal Challeng…",1,0.1,0,0,0.0
1,"""Sunrisers Hyde…","""Royal Challeng…",207,"""Hyderabad""","""Royal Challeng…",1,0.2,0,0,0.0
1,"""Sunrisers Hyde…","""Royal Challeng…",207,"""Hyderabad""","""Royal Challeng…",1,0.3,4,0,4.0
1,"""Sunrisers Hyde…","""Royal Challeng…",207,"""Hyderabad""","""Royal Challeng…",1,0.4,4,0,4.0
1,"""Sunrisers Hyde…","""Royal Challeng…",207,"""Hyderabad""","""Royal Challeng…",1,0.5,6,0,6.0
1,"""Sunrisers Hyde…","""Royal Challeng…",207,"""Hyderabad""","""Royal Challeng…",1,0.6,6,0,6.0
1,"""Sunrisers Hyde…","""Royal Challeng…",207,"""Hyderabad""","""Royal Challeng…",1,0.7,7,0,7.0
1,"""Sunrisers Hyde…","""Royal Challeng…",207,"""Hyderabad""","""Royal Challeng…",1,1.1,8,0,4.0
1,"""Sunrisers Hyde…","""Royal Challeng…",207,"""Hyderabad""","""Royal Challeng…",1,1.2,12,0,6.0


In [151]:
deliveries=deliveries.to_dummies(["batting_team","bowling_team","city","toss_winner"])
deliveries=deliveries.filter(pl.col("balls")>=6)
deliveries

inning,batting_team_Chennai Super Kings,batting_team_Delhi Capitals,batting_team_Gujarat Lions,batting_team_Kings XI Punjab,batting_team_Kolkata Knight Riders,batting_team_Mumbai Indians,batting_team_Pune Warriors,batting_team_Rajasthan Royals,batting_team_Royal Challengers Bangalore,batting_team_Sunrisers Hyderabad,bowling_team_Chennai Super Kings,bowling_team_Delhi Capitals,bowling_team_Gujarat Lions,bowling_team_Kings XI Punjab,bowling_team_Kolkata Knight Riders,bowling_team_Mumbai Indians,bowling_team_Pune Warriors,bowling_team_Rajasthan Royals,bowling_team_Royal Challengers Bangalore,bowling_team_Sunrisers Hyderabad,final_runs,city_Ahmedabad,city_Bangalore,city_Chandigarh,city_Chennai,city_Delhi,city_Dharamsala,city_Hyderabad,city_Jaipur,city_Kolkata,city_Mohali,city_Mumbai,city_Pune,toss_winner_Chennai Super Kings,toss_winner_Delhi Capitals,toss_winner_Gujarat Lions,toss_winner_Kings XI Punjab,toss_winner_Kolkata Knight Riders,toss_winner_Mumbai Indians,toss_winner_Pune Warriors,toss_winner_Rajasthan Royals,toss_winner_Royal Challengers Bangalore,toss_winner_Sunrisers Hyderabad,toss_decision,balls,current_runs,wickets,current_run_rate
i64,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,i64,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,i64,f64,i64,i64,f64
1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,207,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,6.1,60,1,8.571429
1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,207,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,6.2,60,1,8.571429
1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,207,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,6.3,61,1,8.714286
1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,207,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,6.4,62,1,8.857143
1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,207,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,6.5,63,1,9.0
1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,207,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,6.6,64,1,9.142857
1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,207,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,7.1,65,1,8.125
1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,207,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,7.2,66,1,8.25
1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,207,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,7.3,67,1,8.375


In [152]:
deliveries=deliveries.to_pandas()

In [153]:
deliveries=deliveries[deliveries["balls"]>=6]
deliveries

,inning,batting_team_Chennai Super Kings,batting_team_Delhi Capitals,batting_team_Gujarat Lions,batting_team_Kings XI Punjab,batting_team_Kolkata Knight Riders,batting_team_Mumbai Indians,batting_team_Pune Warriors,batting_team_Rajasthan Royals,batting_team_Royal Challengers Bangalore,...,toss_winner_Mumbai Indians,toss_winner_Pune Warriors,toss_winner_Rajasthan Royals,toss_winner_Royal Challengers Bangalore,toss_winner_Sunrisers Hyderabad,toss_decision,balls,current_runs,wickets,current_run_rate
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,6.1,60,1,8.571429
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,6.2,60,1,8.571429
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,6.3,61,1,8.714286
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,6.4,62,1,8.857143
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,6.5,63,1,9.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88172,2,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,19.2,152,5,7.600000
88173,2,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,19.3,154,5,7.700000
88174,2,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,19.4,155,6,7.750000
88175,2,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,19.5,157,6,7.850000


In [154]:
deliveries.to_csv("ipl_final_data.csv")

In [155]:
s=setup(deliveries,target='final_runs')

,Description,Value
0,Session id,3845
1,Target,final_runs
2,Target type,Regression
3,Original data shape,"(88177, 49)"
4,Transformed data shape,"(88177, 49)"
5,Transformed train set shape,"(61723, 49)"
6,Transformed test set shape,"(26454, 49)"
7,Numeric features,48
8,Preprocess,True
9,Imputation type,simple


In [156]:
best=compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,1.7495,22.8481,4.7749,0.9742,0.0306,0.0111,8.2340
rf,Random Forest Regressor,3.2949,42.2392,6.4962,0.9523,0.0425,0.0213,6.9550
dt,Decision Tree Regressor,2.6201,82.5901,9.0826,0.9067,0.0587,0.0167,0.7790
lightgbm,Light Gradient Boosting Machine,10.2226,186.6235,13.6602,0.7894,0.0909,0.0675,0.7790
knn,K Neighbors Regressor,9.5427,199.9448,14.1386,0.7743,0.0922,0.0619,1.1080
gbr,Gradient Boosting Regressor,12.9587,304.6243,17.4524,0.6562,0.1177,0.0870,2.3200
lr,Linear Regression,13.9524,348.8354,18.6764,0.6063,0.1267,0.0941,1.2650
ridge,Ridge Regression,13.9521,348.8281,18.6762,0.6063,0.1267,0.0941,0.7170
br,Bayesian Ridge,13.9486,348.8254,18.6761,0.6063,0.1267,0.0941,0.8190
omp,Orthogonal Matching Pursuit,14.2577,367.6325,19.1730,0.5851,0.1296,0.0961,0.6310


In [157]:
evaluate_model(best)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [158]:
test=predict_model(best)
test

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Extra Trees Regressor,1.6037,22.2590,4.7179,0.9747,0.0298,0.0102


,inning,batting_team_Chennai Super Kings,batting_team_Delhi Capitals,batting_team_Gujarat Lions,batting_team_Kings XI Punjab,batting_team_Kolkata Knight Riders,batting_team_Mumbai Indians,batting_team_Pune Warriors,batting_team_Rajasthan Royals,batting_team_Royal Challengers Bangalore,...,toss_winner_Rajasthan Royals,toss_winner_Royal Challengers Bangalore,toss_winner_Sunrisers Hyderabad,toss_decision,balls,current_runs,wickets,current_run_rate,final_runs,prediction_label
15480,1,0,0,0,0,0,1,0,0,0,...,0,0,0,1,12.2,127,3,9.769231,218,217.880
16622,1,0,0,0,0,0,0,0,1,0,...,0,0,0,1,16.4,145,3,8.529411,183,183.410
74360,2,0,0,0,0,0,0,0,1,0,...,0,0,1,0,14.5,102,4,6.800000,141,141.550
82475,2,0,0,0,1,0,0,0,0,0,...,0,0,0,1,15.6,120,1,7.500000,152,151.680
48237,1,0,0,0,0,0,0,0,0,1,...,0,0,0,1,6.6,66,0,9.428572,190,190.360
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9567,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,8.1,50,3,5.555555,137,136.830
9204,2,1,0,0,0,0,0,0,0,0,...,0,0,0,0,16.1,143,1,8.411765,152,153.170
16067,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,7.5,61,2,7.625000,151,161.070
3261,1,0,0,0,0,1,0,0,0,0,...,0,1,0,1,17.6,125,9,6.944445,131,132.990


In [159]:
test.iloc[2:30]

,inning,batting_team_Chennai Super Kings,batting_team_Delhi Capitals,batting_team_Gujarat Lions,batting_team_Kings XI Punjab,batting_team_Kolkata Knight Riders,batting_team_Mumbai Indians,batting_team_Pune Warriors,batting_team_Rajasthan Royals,batting_team_Royal Challengers Bangalore,...,toss_winner_Rajasthan Royals,toss_winner_Royal Challengers Bangalore,toss_winner_Sunrisers Hyderabad,toss_decision,balls,current_runs,wickets,current_run_rate,final_runs,prediction_label
74360,2,0,0,0,0,0,0,0,1,0,...,0,0,1,0,14.500000,102,4,6.800000,141,141.55
82475,2,0,0,0,1,0,0,0,0,0,...,0,0,0,1,15.600000,120,1,7.500000,152,151.68
48237,1,0,0,0,0,0,0,0,0,1,...,0,0,0,1,6.600000,66,0,9.428572,190,190.36
42733,2,1,0,0,0,0,0,0,0,0,...,1,0,0,0,14.200000,130,2,8.666667,186,185.87
28080,1,0,0,0,0,1,0,0,0,0,...,0,0,0,1,9.600000,49,4,4.900000,147,146.82
50248,2,0,0,0,0,0,0,0,0,1,...,0,1,0,1,14.400000,95,5,6.333333,158,155.15
82335,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,6.400000,37,1,5.285714,158,158.00
30163,1,0,0,0,0,0,0,0,0,1,...,0,1,0,0,11.300000,111,1,9.250000,205,205.43
68184,1,0,0,0,0,0,0,0,0,1,...,0,0,0,1,10.600000,79,3,7.181818,175,175.00
23108,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,7.500000,47,4,5.875000,118,118.03


In [160]:
trained_model=create_model(best)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,1.6808,21.9256,4.6825,0.9751,0.0299,0.0106
1,1.7193,19.8773,4.4584,0.9777,0.0284,0.0109
2,1.8003,25.7827,5.0777,0.9708,0.0321,0.0114
3,1.7598,22.1021,4.7013,0.9748,0.0303,0.0112
4,1.7703,21.9389,4.6839,0.9758,0.0299,0.0112
5,1.6941,23.1993,4.8166,0.9737,0.0312,0.0109
6,1.8065,26.0713,5.1060,0.9711,0.0317,0.0113
7,1.8478,25.6396,5.0636,0.9714,0.0338,0.0120
8,1.7276,21.3379,4.6193,0.9758,0.0294,0.0109


In [161]:
from google.colab import drive
drive.mount("/content/drive")

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
save_model(trained_model,"/content/drive/MyDrive/ipl_colab_model")

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/ipl_colab_model.pkl'

In [171]:
save_model(best,"ipl_pycaret")

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=FastMemory(location=C:\Users\sorra\AppData\Local\Temp\joblib),
          steps=[('numerical_imputer',
                  TransformerWrapper(include=['inning',
                                              'batting_team_Chennai Super Kings',
                                              'batting_team_Delhi Capitals',
                                              'batting_team_Gujarat Lions',
                                              'batting_team_Kings XI Punjab',
                                              'batting_team_Kolkata Knight '
                                              'Riders',
                                              'batting_team_Mumbai Indians',
                                              'batting_team_Pune Warr...
                                              'city_Delhi', 'city_Dharamsala',
                                              'city_Hyderabad', 'city_Jaipur',
                                              'city_Kolkata', ...],
   

In [163]:
m=load_model("ipl_pycaret")
m

Transformation Pipeline and Model Successfully Loaded


Pipeline(memory=FastMemory(location=C:\Users\sorra\AppData\Local\Temp\joblib),
         steps=[('numerical_imputer',
                 TransformerWrapper(include=['inning',
                                             'batting_team_Chennai Super Kings',
                                             'batting_team_Delhi Capitals',
                                             'batting_team_Gujarat Lions',
                                             'batting_team_Kings XI Punjab',
                                             'batting_team_Kolkata Knight '
                                             'Riders',
                                             'batting_team_Mumbai Indians',
                                             'batting_team_Pune Warr...
                                             'city_Delhi', 'city_Dharamsala',
                                             'city_Hyderabad', 'city_Jaipur',
                                             'city_Kolkata', ...],
                                    transformer=SimpleImputer())),
                ('categorical_imputer',
                 TransformerWrapper(include=[],
                                    transformer=SimpleImputer(strategy='most_frequent'))),
                ('clean_column_names',
                 TransformerWrapper(transformer=CleanColumnNames())),
                ('trained_model',
                 ExtraTreesRegressor(n_jobs=-1, random_state=3845))])

In [164]:
test=test[test["balls"]>=6]
test

,inning,batting_team_Chennai Super Kings,batting_team_Delhi Capitals,batting_team_Gujarat Lions,batting_team_Kings XI Punjab,batting_team_Kolkata Knight Riders,batting_team_Mumbai Indians,batting_team_Pune Warriors,batting_team_Rajasthan Royals,batting_team_Royal Challengers Bangalore,...,toss_winner_Rajasthan Royals,toss_winner_Royal Challengers Bangalore,toss_winner_Sunrisers Hyderabad,toss_decision,balls,current_runs,wickets,current_run_rate,final_runs,prediction_label
15480,1,0,0,0,0,0,1,0,0,0,...,0,0,0,1,12.2,127,3,9.769231,218,217.880
16622,1,0,0,0,0,0,0,0,1,0,...,0,0,0,1,16.4,145,3,8.529411,183,183.410
74360,2,0,0,0,0,0,0,0,1,0,...,0,0,1,0,14.5,102,4,6.800000,141,141.550
82475,2,0,0,0,1,0,0,0,0,0,...,0,0,0,1,15.6,120,1,7.500000,152,151.680
48237,1,0,0,0,0,0,0,0,0,1,...,0,0,0,1,6.6,66,0,9.428572,190,190.360
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9567,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,8.1,50,3,5.555555,137,136.830
9204,2,1,0,0,0,0,0,0,0,0,...,0,0,0,0,16.1,143,1,8.411765,152,153.170
16067,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,7.5,61,2,7.625000,151,161.070
3261,1,0,0,0,0,1,0,0,0,0,...,0,1,0,1,17.6,125,9,6.944445,131,132.990


In [165]:
X=test.drop("final_runs",axis=1).drop("prediction_label",axis=1)
Y=test["final_runs"]

In [166]:
X

,inning,batting_team_Chennai Super Kings,batting_team_Delhi Capitals,batting_team_Gujarat Lions,batting_team_Kings XI Punjab,batting_team_Kolkata Knight Riders,batting_team_Mumbai Indians,batting_team_Pune Warriors,batting_team_Rajasthan Royals,batting_team_Royal Challengers Bangalore,...,toss_winner_Mumbai Indians,toss_winner_Pune Warriors,toss_winner_Rajasthan Royals,toss_winner_Royal Challengers Bangalore,toss_winner_Sunrisers Hyderabad,toss_decision,balls,current_runs,wickets,current_run_rate
15480,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,12.2,127,3,9.769231
16622,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,16.4,145,3,8.529411
74360,2,0,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,14.5,102,4,6.800000
82475,2,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,15.6,120,1,7.500000
48237,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,6.6,66,0,9.428572
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9567,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,8.1,50,3,5.555555
9204,2,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,16.1,143,1,8.411765
16067,1,0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,7.5,61,2,7.625000
3261,1,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,1,17.6,125,9,6.944445


In [167]:
test.iloc[1:2].to_numpy()

array([[  1.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   1.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   1.        ,
         16.39999962, 145.        ,   3.        ,   8.52941132,
        183.        , 183.41      ]])

In [168]:
test_score=m.score(X,Y)
test_score

0.974674331759739

In [169]:
X=deliveries.drop("final_runs",axis=1)
Y=deliveries["final_runs"]

In [170]:
train_score=m.score(X,Y)
train_score

0.9920873177468562